# Efficiency Simulation

In [1]:
from __future__ import annotations
from dataclasses import dataclass
from coordinates import Location, Checkpoint, device
import math as m
import copy
from collections.abc import Callable

from torch import FloatTensor, IntTensor, tensor
import torch
import torch.autograd.profiler as profiler
from tqdm.auto import tqdm

In [2]:
from graph import Graph, Node, Transition


In [3]:
import pandas as pd
from coordinates import Checkpoint, Location, device
from torch import tensor

track_data = pd.read_csv("./sem_2023_us.csv")

track_data = track_data.rename(columns={
    "Metres above sea level": "Altitude"
})

track_data.head(10)

checkpoints: list[Checkpoint] = []
for i, row in track_data.iterrows():
    location = Location(tensor(row["Latitude"], device=device), tensor(row["Longitude"], device=device), tensor(row["Altitude"], device=device))
    checkpoints.append(Checkpoint(location, location))

print(f"Found {len(checkpoints)} checkpoints")

Found 2696 checkpoints


In [4]:
def get_coefficient_of_drag(bearing: float) -> float:
    return 0.33

def get_projected_area(bearing: float) -> float:
    """Returns in mm^2"""
    return 943416

g = Graph.construct(
    checkpoints=checkpoints[:5],
    n_points_per_checkpoint=1,
    max_velocity=42 * 1000 / 3600,  # Max velocity the car is allowed to go is 42 km/h
    velocity_step_size=4000 / 3600,
    max_motor_velocity=40 * 1000 / 3600,  # Max velocity the motor is allowed to go is 40 km/h
    motor_velocity_step_size=4000 / 3600,
    wind_velocity=5000 / 3600,
    wind_bearing=200,
    mass=1000,
    coefficient_of_friction=0.03, # TODO: Figure this out
    get_coefficient_of_drag=get_coefficient_of_drag,
    get_projected_area=get_projected_area,
)

0it [00:00, ?it/s]

Processing checkpoint 0. Layer size: 1



100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 17.43it/s]


Processing checkpoint 1. Layer size: 121



100%|█████████████████████████████████████████| 121/121 [00:04<00:00, 25.17it/s]
2it [00:04,  2.45s/it]

Processing checkpoint 2. Layer size: 121



100%|█████████████████████████████████████████| 121/121 [00:05<00:00, 22.49it/s]
3it [00:10,  3.68s/it]

Processing checkpoint 3. Layer size: 121



100%|████████████████████████████████████████| 121/121 [00:00<00:00, 717.64it/s]
4it [00:10,  2.62s/it]


In [5]:
def get_cheapest(g):
    cursor = g.start
    
    path = []

    print(cursor.id)
    
    while cursor is not None:
        print(cursor.id)
        path.append(cursor)

        print(f"L {len(path)}")
        
        target = None
        cheapest = float('inf')
        for t in tqdm(cursor.transitions):
            print(t.work_required)
            if t.work_required < cheapest:
                print("Found cheaper!")
                target = t
                cheapest = t.work_required
        
        cursor = target
        print(cursor)
        
    return path

path = get_cheapest(g)

print("Cheapest path:")
for node in path:
    print(node.id)

0
0
L 1


  0%|          | 0/121 [00:00<?, ?it/s]

tensor(nan, device='cuda:0', dtype=torch.float64)
tensor(nan, device='cuda:0', dtype=torch.float64)
tensor(nan, device='cuda:0', dtype=torch.float64)
tensor(nan, device='cuda:0', dtype=torch.float64)
tensor(nan, device='cuda:0', dtype=torch.float64)
tensor(nan, device='cuda:0', dtype=torch.float64)
tensor(nan, device='cuda:0', dtype=torch.float64)
tensor(nan, device='cuda:0', dtype=torch.float64)
tensor(nan, device='cuda:0', dtype=torch.float64)
tensor(nan, device='cuda:0', dtype=torch.float64)
tensor(nan, device='cuda:0', dtype=torch.float64)
tensor(nan, device='cuda:0', dtype=torch.float64)
tensor(nan, device='cuda:0', dtype=torch.float64)
tensor(nan, device='cuda:0', dtype=torch.float64)
tensor(nan, device='cuda:0', dtype=torch.float64)
tensor(nan, device='cuda:0', dtype=torch.float64)
tensor(nan, device='cuda:0', dtype=torch.float64)
tensor(nan, device='cuda:0', dtype=torch.float64)
tensor(nan, device='cuda:0', dtype=torch.float64)
tensor(nan, device='cuda:0', dtype=torch.float64)


In [6]:
# from typing import List

# def find_cheapest_path(graph: Graph) -> tuple[float, List[Node]]:
#     """Finds the cheapest path from start to end using Dijkstra's algorithm."""
    
#     unvisited_nodes = graph.get_nodes()
#     print(f"Got Nodes: {len(unvisited_nodes)}")

#     costs = dict()
#     paths = dict()
    
#     for node in unvisited_nodes:
#         costs[node] = float('inf')
#         paths[node] = []

#     costs[graph.start] = 0
#     paths[graph.start] = [graph.start]
    
#     cursor = graph.start

#     while cursor is not None:
#         unvisited_nodes.remove(cursor)
        
#         for transition in cursor.transitions:
#             cost = costs[cursor] + transition.work_required
#             if costs[transition.target] > cost:
#                 costs[transition.target] = cost
#                 paths[transition.target] = paths[cursor] + [transition.target]

#         cheapest_node = None
#         cheapest_cost = float('inf')
#         for node, cost in enumerate(costs):
#             if node in unvisited_nodes and cost < cheapest_cost:
#                 cheapest_node = node
#                 cheapest_cost = cost

#         cursor = cheapest_node

#     return costs[graph.end], paths[graph.end]

        


In [7]:
# cost, cheapest_path = find_cheapest_path(g)

# print(f"Cost: {cost}")

In [8]:
# print([(node.id, node.velocity) for node in cheapest_path])

In [9]:
# print(g.get_nodaes())